In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.types import StringType,DoubleType,FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import regexp_replace
import locale
import sys
import os
import re

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
spark

# Leyendo lista de Colegios y Estudiantes

In [4]:
base = '../Colegios'
base2 = '../SB'
colegios = os.listdir('../Colegios')
estudiantes = os.listdir('../SB')

In [5]:
estudiantes.sort()
colegios.sort()
estuDic = {}
coleDic = {}
for estu in estudiantes:
    estuDic[re.findall('\d+', estu)[1]] = estu

for colegio in colegios:
    coleDic[re.findall('\d+', colegio)[1]] = colegio

In [6]:
dfestu = {}
dfcole = {}
for i in coleDic:
    dfestu[i] = None
    dfcole[i] = None

In [7]:
# Funciòn para unir cualquier nùmero de dataframes con la misma estructura 
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [8]:
print("Archivos de estudiantes: ", estuDic['20061'],estuDic['20062'])
print("Archivos de colegios: ",coleDic['2006'])

Archivos de estudiantes:  SB11_20061.txt SB11_20062.txt
Archivos de colegios:  SB11-CLASIFI-PLANTELES-2006.txt


In [9]:
# SE leen los dos archivos y se crean dos dataframes
df1 = spark.read.load(base2 + '/' + estuDic['20061'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬')
df2 = spark.read.load(base2 + '/' + estuDic['20062'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬')

In [10]:
columnasPuntos = [
 'punt_lenguaje',
 'punt_matematicas',
 'punt_c_sociales',
 'punt_filosofia',
 'punt_biologia',
 'punt_quimica',
 'punt_fisica',
 'punt_idioma',
 'punt_profundizacion',
 'punt_interdisciplinar']

In [11]:
# SE unen los dos dataframes
df = unionAll(df1,df2)
for cp in columnasPuntos:
    df = df.withColumn(cp, regexp_replace(cp, ',', '.'))
    df = df.withColumn(cp, df[cp].cast("float"))
    df = df.na.fill({cp:0})

df = df.na.fill({'cole_cod_dane_institucion':'--'})
df = df.na.fill({'cole_nombre_sede':'--'})

df1 = df2 = None

In [12]:

# SE evalua sí tiene valores nullos
print("Nùmero de filas con cole_cod_dane_institucion nulos: ", df.filter("cole_cod_dane_institucion = '--'").count())
print("Nùmero de filas con cole_nombre_sede nulos: ", df.filter("cole_nombre_sede = '--'").count())

NameError: name 'col' is not defined

In [217]:
df.columns

['estu_exam_nombreexamen',
 'estu_estudiante',
 'periodo',
 'estu_consecutivo',
 'estu_edad',
 'estu_tipo_documento',
 'estu_pais_reside',
 'estu_genero',
 'estu_nacimiento_dia',
 'estu_nacimiento_mes',
 'estu_nacimiento_anno',
 'estu_cod_reside_mcpio',
 'estu_reside_mcpio',
 'estu_reside_depto',
 'estu_zona_reside',
 'estu_razoninstituto',
 'cole_valor_pension',
 'estu_ies_cod_deseada',
 'estu_carrdeseada_razon',
 'estu_carrdeseada_cod',
 'estu_ies_deseada_nombre',
 'estu_ies_cod_mpio_deseada',
 'estu_ies_mpio_deseada',
 'estu_ies_dept_deseada',
 'estu_mpio_presentacion',
 'estu_dept_presentacion',
 'estu_exam_cod_mpio_presentacio',
 'estu_ano_termino_bachill',
 'cole_cod_icfes',
 'cole_cod_dane_institucion',
 'cole_nombre_sede',
 'cole_cod_mcpio_ubicacion',
 'cole_calendario',
 'cole_genero',
 'cole_naturaleza',
 'cole_bilingue',
 'cole_jornada',
 'cole_caracter',
 'punt_lenguaje',
 'punt_matematicas',
 'punt_c_sociales',
 'punt_filosofia',
 'punt_biologia',
 'punt_quimica',
 'punt_f

In [218]:
# Contando el numero de estudiantes por id instituciòn y mostrandola de forma descendente
# dsCountEst = df.groupBy('cole_cod_dane_institucion').count().orderBy('count',ascending=False)

# Contando el numero de estudiantes por nombre de instituciòn y mostrandola de forma descendente
# por que hay muchos colegios con cole_cod_dane_institucion es null.
dsCountEst = df.groupBy('cole_nombre_sede').count().orderBy('count',ascending=False)

In [219]:
dsCountEst = dsCountEst.withColumnRenamed("count","num_est")
dsCountEst.show(10,False)

+-------------------------------------------------------------------------------------+-------+
|cole_nombre_sede                                                                     |num_est|
+-------------------------------------------------------------------------------------+-------+
|INST EDUC CEFA                                                                       |1200   |
|INEM FRANCISCO DE PAULA SANTANDER                                                    |987    |
|INSTITUTO TECNICO INDUSTRIAL                                                         |922    |
| COL FUND STA ISABEL DE HUNGRIA                                                      |843    |
|INEM SANTIAGO PEREZ                                                                  |811    |
|INST EDUC INEM JOSE FELIX DE RESTREPO                                                |798    |
|JORGE ELIECER GAITAN                                                                 |708    |
|INSTITUCION EDUCATIVA JOSE MANUEL RODRI

In [220]:
rowmax = dsCountEst.agg({"num_est": "max"}).collect()[0]
rowmin = dsCountEst.agg({"num_est": "min"}).collect()[0]

num_est_max = rowmax["max(num_est)"]
num_est_min = rowmin["min(num_est)"]

print("Numero màximo de estudiantes: ",num_est_max)
print("Numero mìnimo de estudiantes: ",num_est_min)

Numero màximo de estudiantes:  1200
Numero mìnimo de estudiantes:  1


In [221]:
#Se crea una columna con el total de los puntajes   
df= df.withColumn("Suma",df['punt_lenguaje']+df['punt_matematicas']+df['punt_c_sociales']+
                     df['punt_filosofia']+df['punt_biologia']+df['punt_quimica']+df['punt_fisica']+
                     df['punt_idioma']+df['punt_profundizacion']+df['punt_interdisciplinar'])

In [222]:
#Se crea una lista de columnas para hacer el analisis.
columnasToAnalysis = columnasPuntos.copy()
columnasToAnalysis.insert(0,'cole_cod_dane_institucion')
columnasToAnalysis.insert(1,'cole_nombre_sede')
columnasToAnalysis.append('Suma')

In [223]:
columnasToAnalysis

['cole_cod_dane_institucion',
 'cole_nombre_sede',
 'punt_lenguaje',
 'punt_matematicas',
 'punt_c_sociales',
 'punt_filosofia',
 'punt_biologia',
 'punt_quimica',
 'punt_fisica',
 'punt_idioma',
 'punt_profundizacion',
 'punt_interdisciplinar',
 'Suma']

In [224]:
# SE Selecciona el subgrupo de informciòn que se analizara
df = df.select(columnasToAnalysis)

In [225]:
df.columns

['cole_cod_dane_institucion',
 'cole_nombre_sede',
 'punt_lenguaje',
 'punt_matematicas',
 'punt_c_sociales',
 'punt_filosofia',
 'punt_biologia',
 'punt_quimica',
 'punt_fisica',
 'punt_idioma',
 'punt_profundizacion',
 'punt_interdisciplinar',
 'Suma']

## Resultados finales por instituciòn

In [226]:
# Còdigo para obtener la media de los resultados de las evaluaciones de todos los estudiantes por instituciòn 
# y posiciòn en orden descendente.
from pyspark.sql.functions import *
from pyspark.sql.window import Window

#df = df.groupBy('cole_cod_dane_institucion','cole_nombre_sede').mean()
df = df.groupBy('cole_nombre_sede').mean()

rowprommax = df.agg({"avg(Suma)": "max"}).collect()[0]
rowprommin = df.agg({"avg(Suma)": "min"}).collect()[0]

prom_est_max = rowprommax["max(avg(Suma))"]
prom_est_min = rowprommin["min(avg(Suma))"]

print("Màxima calificaciòn en colegios: ", prom_est_max)
print("Mìxima calificaciòn en colegios: ", prom_est_min)

#df = df.withColumn("rank", dense_rank().over(Window.orderBy(desc("avg(Suma)"))))
#df = df.orderBy('rank',ascending= True)

Màxima calificaciòn en colegios:  590.5481338500977
Mìxima calificaciòn en colegios:  333.7709017666903


## Creación de Indices de evaluación
$Indice\_Prom = \frac{avg(Suma)}{max(avg(Suma))}$&nbsp;&nbsp;
$0 < Indice\_Prom \leq 1$

$Indice\_Num = \frac{num\_est}{max(num\_est)}$&nbsp;&nbsp;
$0 < Indice\_Num \leq 1$

$Indice\_Total = Indice\_Prom \times Indice\_Num$&nbsp;&nbsp;
$0 < Indice\_Total \leq 1 $

$Indice\_Total$: tiene en consideraciòn el promedio de las calificaciones y el nùmero de estudiantes 

In [227]:
dfjoin = df.join(dsCountEst,"cole_nombre_sede")
dfjoin= dfjoin.withColumn("Indice_Prom",round(dfjoin['avg(Suma)']/prom_est_max,3))
dfjoin= dfjoin.withColumn("Indice_Num",round(dfjoin['num_est']/num_est_max,3))
dfjoin= dfjoin.withColumn("Indice_Total",round(dfjoin['Indice_Prom']*dfjoin['Indice_Num'],3))

In [228]:
dfjoin.printSchema()

root
 |-- cole_nombre_sede: string (nullable = true)
 |-- avg(punt_lenguaje): double (nullable = true)
 |-- avg(punt_matematicas): double (nullable = true)
 |-- avg(punt_c_sociales): double (nullable = true)
 |-- avg(punt_filosofia): double (nullable = true)
 |-- avg(punt_biologia): double (nullable = true)
 |-- avg(punt_quimica): double (nullable = true)
 |-- avg(punt_fisica): double (nullable = true)
 |-- avg(punt_idioma): double (nullable = true)
 |-- avg(punt_profundizacion): double (nullable = true)
 |-- avg(punt_interdisciplinar): double (nullable = true)
 |-- avg(Suma): double (nullable = true)
 |-- num_est: long (nullable = false)
 |-- Indice_Prom: double (nullable = true)
 |-- Indice_Num: double (nullable = true)
 |-- Indice_Total: double (nullable = true)



In [229]:
dfjoin = dfjoin.select('cole_nombre_sede','avg(Suma)','num_est','Indice_Prom','Indice_Num','Indice_Total')
dfjoin = dfjoin.withColumn("rankProm", dense_rank().over(Window.orderBy(desc("Indice_Prom"))))
dfjoin = dfjoin.withColumn("rankNum", dense_rank().over(Window.orderBy(desc("Indice_Num"))))
dfjoin = dfjoin.withColumn("rankTotal", dense_rank().over(Window.orderBy(desc("Indice_Total"))))
dfjoin = dfjoin.withColumn("Media",round(dfjoin['avg(Suma)'],2))
dfjoin.printSchema()

root
 |-- cole_nombre_sede: string (nullable = true)
 |-- avg(Suma): double (nullable = true)
 |-- num_est: long (nullable = false)
 |-- Indice_Prom: double (nullable = true)
 |-- Indice_Num: double (nullable = true)
 |-- Indice_Total: double (nullable = true)
 |-- rankProm: integer (nullable = true)
 |-- rankNum: integer (nullable = true)
 |-- rankTotal: integer (nullable = true)
 |-- Media: double (nullable = true)



In [230]:
# Se ordena por Indice_Prom
dfjoin = dfjoin.orderBy('rankProm',ascending= True)
dfjoin.select('cole_nombre_sede','Media','Indice_Prom','num_est','rankProm','rankNum','rankTotal').show(10)

+--------------------+------+-----------+-------+--------+-------+---------+
|    cole_nombre_sede| Media|Indice_Prom|num_est|rankProm|rankNum|rankTotal|
+--------------------+------+-----------+-------+--------+-------+---------+
|INST ALBERTO MERA...|590.55|        1.0|     16|       1|    324|      250|
|LIC NAVARRA      ...|572.34|      0.969|     36|       2|    307|      234|
|COL LOS NOGALES  ...|568.15|      0.962|     39|       3|    304|      231|
|LIC  DE CERVANTES...|566.35|      0.959|     96|       4|    257|      186|
|LIC CAMPO DAVID  ...|563.75|      0.955|     19|       5|    321|      248|
|COL SAN CARLOS   ...|563.34|      0.954|     94|       6|    259|      189|
|COLEGIO LEONARDO ...|562.99|      0.953|      4|       7|    334|      260|
|COLEGIO MARYMOUNT...|563.02|      0.953|     97|       7|    256|      186|
|COL  SAN BONIFACI...|562.43|      0.952|     18|       8|    322|      249|
|COL SANTA FRANCIS...|559.54|      0.947|     49|       9|    296|      224|

In [231]:
dfjoin = dfjoin.orderBy('rankNum',ascending= True)
dfjoin.select('cole_nombre_sede','Media','Indice_Num','num_est','rankProm','rankNum','rankTotal').show(10)

+--------------------+------+----------+-------+--------+-------+---------+
|    cole_nombre_sede| Media|Indice_Num|num_est|rankProm|rankNum|rankTotal|
+--------------------+------+----------+-------+--------+-------+---------+
|      INST EDUC CEFA|427.16|       1.0|   1200|     211|      1|        1|
|INEM FRANCISCO DE...|425.62|     0.823|    987|     213|      2|        2|
|INSTITUTO TECNICO...|422.39|     0.768|    922|     219|      3|        3|
| COL FUND STA ISA...|412.87|     0.703|    843|     235|      4|        4|
| INEM SANTIAGO PEREZ|418.96|     0.676|    811|     225|      5|        5|
|INST EDUC INEM JO...|436.27|     0.665|    798|     195|      6|        4|
|JORGE ELIECER GAITAN|404.36|      0.59|    708|     249|      7|        7|
|INSTITUCION EDUCA...|409.09|     0.585|    702|     241|      8|        6|
|INSTITUCION EDUCA...|402.18|     0.527|    632|     253|      9|       10|
|INSTITUTO REGIONA...|399.98|     0.523|    627|     257|     10|       11|
+-----------

In [232]:
# Se ordena por Indice_Total
dfjoin = dfjoin.orderBy('rankTotal',ascending= True)
dfjoin.select('cole_nombre_sede','Media','Indice_Total','num_est','rankProm','rankNum','rankTotal').show(10)

+--------------------+------+------------+-------+--------+-------+---------+
|    cole_nombre_sede| Media|Indice_Total|num_est|rankProm|rankNum|rankTotal|
+--------------------+------+------------+-------+--------+-------+---------+
|      INST EDUC CEFA|427.16|       0.723|   1200|     211|      1|        1|
|INEM FRANCISCO DE...|425.62|       0.593|    987|     213|      2|        2|
|INSTITUTO TECNICO...|422.39|       0.549|    922|     219|      3|        3|
| COL FUND STA ISA...|412.87|       0.491|    843|     235|      4|        4|
|INST EDUC INEM JO...|436.27|       0.491|    798|     195|      6|        4|
| INEM SANTIAGO PEREZ|418.96|       0.479|    811|     225|      5|        5|
|INSTITUCION EDUCA...|409.09|       0.405|    702|     241|      8|        6|
|JORGE ELIECER GAITAN|404.36|       0.404|    708|     249|      7|        7|
|LIC SALAZAR Y HER...|454.56|       0.395|    616|     164|     11|        8|
|COL DE LA UNIV PO...|490.85|       0.387|    559|     103|     

## Colegios 2006

In [26]:
# se crea un dataframe con una columna que lo señala el año 2006.
from pyspark.sql.functions import lit
df = df.withColumn('año',lit(2006))
df.show(5,False)

+-------------------------+------------------+---------------------+--------------------+-------------------+------------------+------------------+-----------------+-----------------+------------------------+--------------------------+-----------------+----+----+
|cole_cod_dane_institucion|avg(punt_lenguaje)|avg(punt_matematicas)|avg(punt_c_sociales)|avg(punt_filosofia)|avg(punt_biologia)|avg(punt_quimica) |avg(punt_fisica) |avg(punt_idioma) |avg(punt_profundizacion)|avg(punt_interdisciplinar)|avg(Suma)        |rank|año |
+-------------------------+------------------+---------------------+--------------------+-------------------+------------------+------------------+-----------------+-----------------+------------------------+--------------------------+-----------------+----+----+
|311001089221             |65.57875061035156 |65.78999948501587    |69.42500019073486   |61.70125079154968  |69.47937536239624 |61.438124656677246|62.29437446594238|70.63875007629395|6.42750009894371        |

In [27]:
# SE crea Dataframe de colegios 2006
dfc = spark.read.load(base + '/' + coleDic['2006'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='\t')
dfc = dfc.withColumn('COLE_CODIGO_COLEGIO', dfc['COLE_CODIGO_COLEGIO'].cast("string"))
dfc = dfc.withColumn('COLE_CODMPIO_COLEGIO', dfc['COLE_CODMPIO_COLEGIO'].cast("string"))

In [28]:
print(dfc.printSchema())

root
 |-- COLE_CODIGO_COLEGIO: string (nullable = true)
 |-- COLE_INST_NOMBRE: string (nullable = true)
 |-- COLE_CODMPIO_COLEGIO: string (nullable = true)
 |-- COLE_MPIO_COLEGIO: string (nullable = true)
 |-- COLE_DEPTO_COLEGIO: string (nullable = true)
 |-- COLE_INST_JORNADA: string (nullable = true)
 |-- COLE_CALENDARIO_COLEGIO: string (nullable = true)
 |-- COLE_GENEROPOBLACION: string (nullable = true)
 |-- COLE_NATURALEZA: string (nullable = true)
 |-- COLE_CIENCIAS_SOCIALES: integer (nullable = true)
 |-- COLE_QUIMICA: integer (nullable = true)
 |-- COLE_FISICA: integer (nullable = true)
 |-- COLE_BIOLOGIA: integer (nullable = true)
 |-- COLE_FILOSOFIA: integer (nullable = true)
 |-- COLE_MATEMATICAS: integer (nullable = true)
 |-- COLE_LENGUAJE: integer (nullable = true)
 |-- COLE_INGLES: string (nullable = true)
 |-- COLE_GEOGRAFIA: string (nullable = true)
 |-- COLE_HISTORIA: string (nullable = true)
 |-- COLE_CATEGORIA: string (nullable = true)
 |-- COLE_ESTUDIANTES_PRESENTE

In [29]:
dfc.columns

['COLE_CODIGO_COLEGIO',
 'COLE_INST_NOMBRE',
 'COLE_CODMPIO_COLEGIO',
 'COLE_MPIO_COLEGIO',
 'COLE_DEPTO_COLEGIO',
 'COLE_INST_JORNADA',
 'COLE_CALENDARIO_COLEGIO',
 'COLE_GENEROPOBLACION',
 'COLE_NATURALEZA',
 'COLE_CIENCIAS_SOCIALES',
 'COLE_QUIMICA',
 'COLE_FISICA',
 'COLE_BIOLOGIA',
 'COLE_FILOSOFIA',
 'COLE_MATEMATICAS',
 'COLE_LENGUAJE',
 'COLE_INGLES',
 'COLE_GEOGRAFIA',
 'COLE_HISTORIA',
 'COLE_CATEGORIA',
 'COLE_ESTUDIANTES_PRESENTES']

In [30]:
colsColegio = ['COLE_CODIGO_COLEGIO',
 'COLE_CODMPIO_COLEGIO',
 'COLE_CIENCIAS_SOCIALES',
 'COLE_QUIMICA',
 'COLE_FISICA',
 'COLE_BIOLOGIA',
 'COLE_FILOSOFIA',
 'COLE_MATEMATICAS',
 'COLE_LENGUAJE',
 'COLE_INGLES',
 'COLE_GEOGRAFIA',
 'COLE_HISTORIA']

In [31]:
dfc = dfc.select(colsColegio)

In [35]:
dfc.show(5,False)

+-------------------+--------------------+----------------------+------------+-----------+-------------+--------------+----------------+-------------+-----------+--------------+-------------+
|COLE_CODIGO_COLEGIO|COLE_CODMPIO_COLEGIO|COLE_CIENCIAS_SOCIALES|COLE_QUIMICA|COLE_FISICA|COLE_BIOLOGIA|COLE_FILOSOFIA|COLE_MATEMATICAS|COLE_LENGUAJE|COLE_INGLES|COLE_GEOGRAFIA|COLE_HISTORIA|
+-------------------+--------------------+----------------------+------------+-----------+-------------+--------------+----------------+-------------+-----------+--------------+-------------+
|127696             |5001                |6                     |6           |7          |6            |8             |6               |7            |null       |null          |null         |
|127704             |20001               |7                     |7           |7          |7            |8             |6               |7            |null       |null          |null         |
|127712             |41807              

In [36]:
print("El numero de colegios año 2006 es: ", dfc.select('COLE_CODIGO_COLEGIO').distinct().count())

El numero de colegios año 2006 es:  9249


## Anàlisis comparativo entre estudiantes y colegios año 2006

In [38]:
dfjoin2006 = df.join(dfc, df.cole_cod_dane_institucion == dfc.COLE_CODIGO_COLEGIO)

In [92]:
print("Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios:", dfjoin2006.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios: 0


In [39]:
dfjoin2006 = df.join(dfc, df.cole_cod_dane_institucion == dfc.COLE_CODMPIO_COLEGIO)
print("Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios:", dfjoin2006.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios: 0


La informaciòn de colegios no coincide con la informaciòn de estudiantes, no existe cole_cod_dane_institucion en el archivo de colegios.